In [1]:
import os
import sys
import django
import logging
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm
from django.db.models import Count
from sklearn.metrics import mean_squared_error
sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
import django
import time
from main_site.models import Rating
import rs_system.neighborhood_cf_rs as nb_rs

logger = logging.getLogger('Evaluation')

In [17]:
ratings_count = Rating.objects.all().count()
test_data_length = round(ratings_count * 10 /100)
all_ratings_df = pd.DataFrame(list(Rating.objects.all().values()))
logger.debug('{} ratings available'.format(ratings_count))
all_ratings_df.head()
all_ratings_df_columns = list(all_ratings_df.columns.values)
all_ratings_df_columns

2019-07-03 03:05:55,788 : DEBUG : 159 ratings available


['id', 'rating', 'rating_timestamp', 'title_id', 'type', 'user_id']

In [3]:
user_ids = Rating.objects.values('user_id') \
    .annotate(title_count=Count('title_id')) \
    .order_by('-title_count')
title_ids = Rating.objects.values('title_id') \
    .annotate(user_count=Count('user_id')) \
    .order_by('-user_count')
user_ids_df = pd.DataFrame.from_records(user_ids)
title_ids_df = pd.DataFrame.from_records(title_ids)
user_ids_df.head(5)

,title_count,user_id
0,7,39
1,5,48
2,4,26
3,4,122
4,4,45


In [4]:
from sklearn.model_selection import KFold 

In [29]:
kf = KFold(n_splits=10)
for train, test in kf.split(all_ratings_df):
    train_data = np.array(all_ratings_df)[train]
    print("Train_data")
    print(train_data)
    test_data = np.array(all_ratings_df)[test]
    print("Test_data")
    print(test_data)
    print(len(train_data))
    print(len(test_data))
    test_pd = pd.DataFrame(test_data)
    test_pd.columns = all_ratings_df_columns
    print(test_pd)
    print("=====================")
    break
test_pd

Train_data
[[17 Decimal('4.02') Timestamp('2019-07-02 03:58:01.743332') 100
  'calculate' 22]
 [18 Decimal('4.50') Timestamp('2019-07-02 03:58:01.745491') 6
  'calculate' 23]
 [19 Decimal('7.62') Timestamp('2019-07-02 03:58:01.747124') 30
  'calculate' 23]
 [20 Decimal('3.56') Timestamp('2019-07-02 03:58:01.750031') 142
  'calculate' 23]
 [21 Decimal('3.42') Timestamp('2019-07-02 03:58:01.751533') 1
  'calculate' 24]
 [22 Decimal('8.39') Timestamp('2019-07-02 03:58:01.753098') 27
  'calculate' 24]
 [23 Decimal('6.16') Timestamp('2019-07-02 03:58:01.754781') 14
  'calculate' 25]
 [24 Decimal('4.50') Timestamp('2019-07-02 03:58:01.757805') 41
  'calculate' 26]
 [25 Decimal('4.50') Timestamp('2019-07-02 03:58:01.759751') 43
  'calculate' 26]
 [26 Decimal('4.50') Timestamp('2019-07-02 03:58:01.761271') 50
  'calculate' 26]
 [27 Decimal('4.50') Timestamp('2019-07-02 03:58:01.762924') 102
  'calculate' 26]
 [28 Decimal('3.06') Timestamp('2019-07-02 03:58:01.764524') 12
  'calculate' 27]
 [29

,id,rating,rating_timestamp,title_id,type,user_id
0,1,9.43,2019-07-02 03:58:01.701612,20,calculate,8
1,2,9.43,2019-07-02 03:58:01.713435,119,calculate,8
2,3,4.32,2019-07-02 03:58:01.714876,140,calculate,8
3,4,9.08,2019-07-02 03:58:01.717819,113,calculate,9
4,5,9.96,2019-07-02 03:58:01.719302,14,calculate,10
5,6,3.44,2019-07-02 03:58:01.721195,164,calculate,11
6,7,9.52,2019-07-02 03:58:01.722933,143,calculate,13
7,8,4.50,2019-07-02 03:58:01.724418,54,calculate,14
8,9,4.50,2019-07-02 03:58:01.727398,66,calculate,15
9,10,6.99,2019-07-02 03:58:01.729059,88,calculate,16


In [ ]:
# all_ratings_df = pd.merge(all_ratings_df, user_ids_df, left_on='user_id', right_on='user_id')
# all_ratings_df = pd.merge(all_ratings_df, title_ids_df, left_on='title_id', right_on='title_id')
# all_ratings_df.head(5)

In [ ]:
count = 0
test_data = list()
for row in tqdm(user_ids_df.itertuples()):
    if row.title_count == 1:
        break
    if row.title_count > 10:
        limit_row = 3
    elif row.title_count > 5:
        limit_row = 2
    else:
        limit_row = 1
    test_data.extend(list(Rating.objects.filter(user_id=row.user_id).order_by('-rating').values()[:limit_row]))
    count +=limit_row
    if count == test_data_length:
        break
test_data_df =  pd.DataFrame(test_data)
test_data_id = list(test_data_df.id)
test_data_id
# [41, 58, 23, 54, 18, 1, 130, 83, 139, 105, 21, 48, 51, 28

In [ ]:
train_data_df = pd.DataFrame(list(Rating.objects.exclude(id__in=test_data_id).values()))
rating_df = pd.DataFrame(list(Rating.objects.all().values()))
train_data_df

In [ ]:
len(test_data)

In [ ]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)

In [ ]:
train_data_df

In [ ]:
test_data_df

In [ ]:
res_nb = nb_rs.RecommendationNB()
rating_matrix, item_rating_matrix = res_nb.get_rating_matrix()
mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(rating_matrix)
item_mean_centered_rating_matrix = res_nb.get_mean_centered_ratings_matrix(item_rating_matrix)

user_similarity_matrix = res_nb.get_similarity_matrix(rating_matrix)
item_similarity_matrix = res_nb.get_similarity_matrix(item_rating_matrix)

In [ ]:
rating_matrix[8]

In [ ]:
y_true = []
y_pred = []
for row in tqdm(test_data_df.itertuples()):
    y_true.append(float(row.rating))
#     predict_result = res_nb.predict(row.user_id, row.title_id, rating_matrix, mean_centered_rating_matrix, user_similarity_matrix)
    y_pred.append(rating_matrix[row.user_id,row.title_id])
print(y_true)
print(y_pred)

In [33]:
kf = KFold(n_splits=10)
for train, test in kf.split(all_ratings_df):
    # Train data
    train_data = np.array(all_ratings_df)[train]
    train_data_df = pd.DataFrame(train_data)
    train_data_df.columns = all_ratings_df_columns

    # Test data
    test_data = np.array(all_ratings_df)[test]
    test_data_df = pd.DataFrame(test_data)
    test_data_df.columns = all_ratings_df_columns

    # Initialize RS
    res_nb = nb_rs.RecommendationNB(train_data)
    y_true = []
    y_pred = []
    for row in tqdm(test_data_df.itertuples()):
        y_true.append(float(row.rating))
        predict_result = res_nb.predict(row.user_id - 1, row.title_id - 1, res_nb.rating_matrix,
                                            res_nb.mean_centered_ratings_matrix,
                                            res_nb.user_similarity_matrix)
        y_pred.append(predict_result)
    print(y_true)
    print(y_pred)
    print("=====================")
    break

Load all rating data
Creating ratings matrix


AttributeError: 'numpy.ndarray' object has no attribute 'itertuples'